L-Band mixer DB CCA thermal analysis
====
Last update: 31 Aug 2018  

L and S band mixer daughter board CCA dissipate 3.76 watts into the LNB chassis.  Given the air flow, what is the surface temperature of the PCB?  
  
Reference:  
(1) Heat Transfer, A Practical Approach, 2nd Edition, by Yunus Cengel  

In [1]:
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

## Properties of air at 1 atmosphere
Select data taken from Table A-15, page 874.

In [2]:
temp = np.array([-100,-50,-40,-30,-20,-10,0,5,10,15,20,25,30,35,40,45,50,60,70,80,90,100,120],dtype=float)
thermal_conduc = np.array([.01582,.01979,.02057,.02134,.02211,.02288,.02364,.02401,.02439,.02476,.02514,
    .02551,.02588,.02625,.02662,.02699,.02735,.02808,.02881,.02953,.03024,.03095,.03235])
kinem_visc = np.array([.5837,.9319,1.008,1.087,1.169,1.252,1.338,1.382,1.426,1.470,1.516,1.562,1.608,
    1.655,1.702,1.750,1.798,1.896,1.995,2.097,2.201,2.306,2.522])*1e-5
Prandle_num = np.array([.7263,.7440,.7436,.7425,.7408,.7387,.7362,.7350,.7336,.7323,.7309,.7296,.7282,
    .7268,.7255,.7241,.7228,.7202,.7177,.7154,.7132,.7111,.7073])
air_density = np.array([2.038,1.582,1.514,1.451,1.394,1.341,1.292,1.269,1.246,1.225,1.204,1.184,1.164,
    1.145,1.127,1.109,1.092,1.059,1.028,.9994,.9718,.9458,.8977])
spec_heat = np.array([966,999,1002,1004,1005,1006,1006,1006,1006,1007,1007,1007,1007,1007,1007,1007,
    1007,1007,1007,1008,1008,1009,1011],dtype=float)

## Calculate air flow rate
The fans were not specified, for now assume 100 CFM and 17 W per fan and a total of 6 fans per chassis.  Typically 5 inch fans have a CFM range between 80 and 144.  

In [6]:
# Fan specification
CFM_per_fan = 100 # ft^3/min (CFM)
V_dot = CFM_per_fan*0.0283168/60 # convert to m^3/s
print('volume flow rate = {:.3f} m^3/s'.format(V_dot))

T_in = 20  # nominal room temperature, C
Cp = np.interp(T_in,temp,spec_heat) # specific heat of air, see table A-15
rho = np.interp(T_in,temp,air_density) # specific heat of air, see table A-15
print('air density = {:.3f} kg/m^3'.format(rho))

m_dot = rho*V_dot # mass flow rate
print('mass flow rate = {:.3f} kg/s'.format(m_dot))

volume flow rate = 0.047 m^3/s
air density = 1.204 kg/m^3
mass flow rate = 0.057 kg/s


## Calculate the surface temperature of the PCB
The dimensions of the PCBs are estimated.  Assume one fan covers the CCA.

In [8]:
# estimate the size of the PCB
L = 4.2*0.0254  # length of PCB, convert to m
W = 1.23*0.0254  # wide of PCB, convert to m
As = W*L*2 # heat transfer surface area, both sides

# cross section area and hydraulic diameter
Ac = (5*0.0254)**2  #  only look at one fan which forms the 5x5 inch channel
perimeter = (5*0.0254)*4
Dh = 4*Ac/perimeter  # hydraulic diameter, see page 442 in text book
print('hydraulic diameter  = {:.3f} m'.format(Dh))

# velocity of air
vel_air = V_dot/Ac # velocity of air
print('velocity of air  = {:.3f} m/s'.format(vel_air))

# properties of air
v = np.interp(T_in,temp,kinem_visc)  # kinematic viscosity air at 1 atm, see table A-15
Pr = np.interp(T_in,temp,Prandle_num)  # Prandtl number, see table A-15
k = np.interp(T_in,temp,thermal_conduc) # thermal conductivity of air, see table A-15

# calculate Reynolds and Nusselt number, length of hydrodynamic entrance region is ignored
Re = vel_air*Dh/v # Renolds number for air in a rectangular duct
print('Reynolds number = {:,.1f}'.format(Re))
if Re < 2300:
    print('flow is laminar')
    # Nusselt number from Table 15-3, with aspect greater than 8
    Nu = 8.24 # see table 8-1, page 437
if 2300 <= Re and Re < 10000:
    print('flow is transitional')
    # interpolate a Nu value between laminar and turbulent values
    Nu = ((0.023*Re**0.8*Pr**0.4)-8.24)*Re/(2300+10000)+8.24
if Re >= 10000:
    print('flow is turbulent')
    Nu = 0.023*Re**0.8*Pr**0.4  # see equation 8-68, page 441, Dittus-Boelter equation

print('Nusselt number = {:.3f}'.format(Nu))

h = k*Nu/Dh
print('heat transfer coefficient = {:.3f} W/m^2.C'.format(h))

# The temperature difference between the surface of the PCB and the air along the channel.
# Q = h*As*(T_delta)
Q_pcb = 3.76  # power dissipated by PCB
T_delta = Q_pcb/(h*As) # calculate the temperature difference between the surface of the PCB and the air
print('delta temperature  = {:.1f} C'.format(T_delta))
print('PCB surface temperature  = {:.1f} C'.format(T_in+T_delta)) # air at the exit is assumed to be the highest

hydraulic diameter  = 0.127 m
velocity of air  = 2.926 m/s
Reynolds number = 24,512.6
flow is turbulent
Nusselt number = 65.884
heat transfer coefficient = 13.042 W/m^2.C
delta temperature  = 43.3 C
PCB surface temperature  = 63.3 C
